# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.85,83,57,2.56,GS,1709870407
1,1,waitangi,-43.9535,-176.5597,16.01,72,23,1.34,NZ,1709870408
2,2,west island,-12.1568,96.8225,26.99,94,40,5.14,CC,1709870411
3,3,port hedland,-20.3167,118.5667,35.03,49,75,1.54,AU,1709870413
4,4,simpang,-1.2667,104.0833,24.83,92,100,1.86,ID,1709870414


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:


# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      size = "Humidity",
                                      scale = 0.8,
                                      tiles = "OSM"
                                     )

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#Narrow down the city_data_df DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 27 degrees but higher than 21#
#Wind speed less than 4.5 m/s
#Zero cloudiness
city_data_df = city_data_df[(city_data_df['Max Temp']>=21)&(city_data_df['Max Temp']<27)&(city_data_df['Wind Speed']<4.5)&(city_data_df['Cloudiness']==0)]
# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df

/var/folders/qf/cw90bmqj02jdvy10h8g1kvb00000gn/T/ipykernel_23114/1067870840.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,pali,25.7667,73.3333,21.34,25,0,0.87,IN,1709870431
170,170,san pedro pochutla,15.7333,-96.4667,22.15,84,0,2.32,MX,1709870496
245,245,pokhara,28.2333,83.9833,22.09,22,0,2.21,NP,1709870522
273,273,inhambane,-23.8650,35.3833,24.84,90,0,0.67,MZ,1709870527
346,346,tura,25.5198,90.2201,26.25,14,0,2.19,IN,1709870540
352,352,saint-philippe,-21.3585,55.7679,24.77,82,0,3.67,RE,1709870542
353,353,beledweyne,4.7358,45.2036,25.57,81,0,4.30,SO,1709870542
484,484,san luis,-33.2950,-66.3356,21.71,72,0,4.10,AR,1709870509


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Create a dictionary with sample data
data = {
}

# Create the DataFrame
hotel_df = pd.DataFrame(data)

# Display the DataFrame
print(hotel_df)


Empty DataFrame
Columns: []
Index: []


In [7]:
#Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.
hotel_df["City"] = ""
hotel_df["Country"] = ""
hotel_df["Coordinates"] = ""
hotel_df["Humidity"] = ""
hotel_df

,City,Country,Coordinates,Humidity


In [19]:
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Display the new DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity
51,pali,IN,25.7667,73.3333,25
170,san pedro pochutla,MX,15.7333,-96.4667,84
245,pokhara,NP,28.2333,83.9833,22
273,inhambane,MZ,-23.8650,35.3833,90
346,tura,IN,25.5198,90.2201,14
352,saint-philippe,RE,-21.3585,55.7679,82
353,beledweyne,SO,4.7358,45.2036,81
484,san luis,AR,-33.2950,-66.3356,72


In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
51,pali,IN,25.7667,73.3333,25,
170,san pedro pochutla,MX,15.7333,-96.4667,84,
245,pokhara,NP,28.2333,83.9833,22,
273,inhambane,MZ,-23.8650,35.3833,90,
346,tura,IN,25.5198,90.2201,14,
352,saint-philippe,RE,-21.3585,55.7679,82,
353,beledweyne,SO,4.7358,45.2036,81,
484,san luis,AR,-33.2950,-66.3356,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "categories":"accommodation.hotel",
    "filter": "",
    "bias":"",
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
pali - nearest hotel: Hotel Pawan International
san pedro pochutla - nearest hotel: Olimpia
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
inhambane - nearest hotel: Hotel de Inhambane
tura - nearest hotel: Hotel Polo Orchid
saint-philippe - nearest hotel: Le Baril
beledweyne - nearest hotel: Hotel Medina
san luis - nearest hotel: Hotel Dos Venados


,City,Country,Lat,Lng,Humidity,Hotel Name
51,pali,IN,25.7667,73.3333,25,Hotel Pawan International
170,san pedro pochutla,MX,15.7333,-96.4667,84,Olimpia
245,pokhara,NP,28.2333,83.9833,22,Hotel Annapurna View Sarangkot
273,inhambane,MZ,-23.8650,35.3833,90,Hotel de Inhambane
346,tura,IN,25.5198,90.2201,14,Hotel Polo Orchid
352,saint-philippe,RE,-21.3585,55.7679,82,Le Baril
353,beledweyne,SO,4.7358,45.2036,81,Hotel Medina
484,san luis,AR,-33.2950,-66.3356,72,Hotel Dos Venados


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot

map = hotel_df.hvplot.points('Lng',
             'Lat',
             geo=True,
             tiles ='OSM',
             frame_width = 700,
             frame_height = 500,
             color='City',
             alpha=0.6,
             hover='True',
             hover_cols='all',
             use_index=False,
             s='Humidity',
             scale=1.2)


# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)